In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
import urllib.request
import json
from datetime import date

/Users/honzapulpan/.pyenv/versions/3.6.4/envs/jupyter/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
with urllib.request.urlopen("https://api.apify.com/v2/key-value-stores/K373S4uCFR9W1K8ei/records/LATEST?disableRedirect=true") as url:
    data = json.loads(url.read().decode())

In [31]:
with urllib.request.urlopen("https://onemocneni-aktualne.mzcr.cz/api/v1/covid-19/testy.json") as url:
    numberOfTested_json = json.loads(url.read().decode())
    numberOfTested = pd.DataFrame(numberOfTested_json['data'])
    numberOfTested['datum'] = pd.to_datetime(numberOfTested['datum'], errors='coerce')    
    numberOfTested.to_csv('tested-'+numberOfTested_json['data'][-1]['datum']+'.csv', index=False)

In [32]:
with urllib.request.urlopen("https://onemocneni-aktualne.mzcr.cz/api/v1/covid-19/nakaza.json") as url:
    totalPositiveTests_json = json.loads(url.read().decode())
    totalPositiveTests = pd.DataFrame(totalPositiveTests_json)
    totalPositiveTests['datum'] = pd.to_datetime(totalPositiveTests['datum'], errors='coerce')    
    totalPositiveTests.to_csv('positive-'+totalPositiveTests_json[-1]['datum']+'.csv', index=False)    

In [41]:
with urllib.request.urlopen("https://onemocneni-aktualne.mzcr.cz/api/v1/covid-19/osoby.json") as url:
    positiveTests_json = json.loads(url.read().decode())
    positiveTests = pd.DataFrame(positiveTests_json)
    positiveTests['DatumHlaseni'] = pd.to_datetime(positiveTests['DatumHlaseni'], errors='coerce')    
    positiveTests.to_csv('persons-'+str(max(positiveTests['DatumHlaseni'].dt.date))+'.csv', index=False)    
    #display(positiveTests.tail())
    #display()

In [6]:
#TODO
# - upravit tak, aby pracovalo skutečně s datumem. Zatim handluju today jako string

# možná přidávat i absolutní čísla a ty daily přepočítat až na konci pro celej dataset 
# (kvůli tomu, ž datumy nemusí být vždy v řadě)

daily_data = pd.read_csv('daily_data.csv')
#display(daily_data)

today = date.today()
fi = daily_data.index[daily_data['date'] == str(today)]

if len(fi.tolist()):
    print('update')
    daily_data.loc[fi, 
           ['totalTested',
            'tested',
            'totalInfected',
            'infected',
            'totalRecovered',
            'recovered',
            'totalDeceased',
            'deceased']] = data['totalTested'],\
                           data['totalTested'] - daily_data['totalTested'].loc[fi-1],\
                           data['infected'],\
                           data['infected'] - daily_data['totalInfected'].loc[fi-1],\
                           data['recovered'],\
                           data['recovered'] - daily_data['totalRecovered'].loc[fi-1],\
                           data['deceased'],\
                           data['deceased'] - daily_data['totalDeceased'].loc[fi-1]
else:
    print('append')
    daily_data = daily_data.append({'date': today,
                                    'totalTested': data['totalTested'],
                                    'tested': data['totalTested'] - daily_data['totalTested'].iloc[-1],
                                    'totalInfected': data['infected'],
                                    'infected': data['infected'] - daily_data['totalInfected'].iloc[-1],                                    
                                    'totalRecovered': data['recovered'],
                                    'recovered': data['recovered'] - daily_data['totalRecovered'].iloc[-1],
                                    'totalDeceased': data['deceased'],
                                    'deceased': data['deceased'] - daily_data['totalDeceased'].iloc[-1]},
                                    ignore_index=True)                                                                                          
daily_data.to_csv('daily_data.csv', index=False)
display(daily_data)

append


,date,totalTested,tested,totalInfected,infected,totalRecovered,recovered,totalDeceased,deceased
0,2020-03-26,31127,4429,2022,259,10,2,9,3
1,2020-03-27,36374,5247,2395,373,11,1,9,0
2,2020-03-28,40700,4326,2657,262,11,0,11,2
3,2020-03-29,40700,0,2775,118,11,0,16,5
4,2020-03-30,43498,2798,2837,62,11,0,17,1
